## SIRF simulation with Siemens mMR NEMA data

First import data following PETRIC routine

In [2]:
from petric import get_data
from SIRF_data_preparation.dataset_settings import get_settings
import sirf.STIR as pet

scanID = "Siemens_mMR_NEMA_IQ"
settings = get_settings(scanID)
slices = settings.slices
num_subsets = settings.num_subsets
outdir = f"./output/{scanID}/test_PDHG"

#Simulater data (full)
data = get_data('./data/Siemens_mMR_NEMA_IQ', outdir=outdir)


### Set up (full) forward model

Following sirf.contrib.partitioner.partitioner

#### Issue:
Sim data has negative entries

In [4]:
sensitivity_factors = pet.AcquisitionSensitivityModel(data.mult_factors)
sensitivity_factors.set_up(data.mult_factors)
acquisition_model = pet.AcquisitionModelUsingParallelproj()
acquisition_model.set_acquisition_sensitivity(sensitivity_factors)
acquisition_model.set_additive_term(data.additive_term)
acquisition_model.set_up(data.acquired_data, data.OSEM_image)

x_ref = data.reference_image
sim_data = acquisition_model.forward(x_ref)
print('Minimum of sim data: ', sim_data.min())



Minimum of sim data:  -3.1595205e-31


### Set up (full) forward model

We consider the forward model $b=Ax+r$

- $A$: (linear) Forward model 
- $r$: Background 
- $b$: Measurement

#### Questions:
According to SIRF user guide (https://github.com/SyneRBI/SIRF/blob/master/doc/UserGuide.md#pet), the forwad model is given by:
$y=S(GPx+a)+b$

- $S$: Acquisition sensitivity (Modelled with data.mult_factors?)
- $G$: Ray tracing matrix - pet.AcquisitionModelUsingParallelproj()
- $P$: Image data processor (identity in PETRIC?)
- $a$: Additive term (Modelled with data.additive_term?)
- $b$: Background term (0 in PETRIC)

#### To relate the two descriptions:

- $A$=acquisition_model.get_linear_acquisition_model()=$SGP$
- $r$=acquisition_model.get_constant_term()=$Sa+b$




#### Check components 

In [5]:
A = acquisition_model.get_linear_acquisition_model()
b_linear = A.forward(x_ref)
r = acquisition_model.get_constant_term()
print('Minimum of linear data (Ax):', b_linear.min())
print('Minimum of background (r):', r.min())

Minimum of linear data (Ax): -1.7910054e-10
Minimum of background (r): 0.0


#### Inconsistency of linear model?

To also verify the translation from SIRF to mathematical model, we compare simulated data using the whole acquisition model as well as the linear model. 

- b1: Calling whole acquisition model
- b2: Calling linear model, then add 'background'

In [6]:
b1 = acquisition_model.forward(x_ref)
b2 = A.forward(x_ref)+r
diff = b1-b2
print('Norm of diff:', diff.norm())
print('Max of diff:', diff.max())
print('Min of diff:', diff.min())

Norm of diff: 0.00032855209428817034
Max of diff: 4.7683716e-07
Min of diff: -4.7683716e-07
